# 使用飞书知识库实现查询

直接用request不好，放弃，改用官方python sdk，另外起新的页面。

## 全局设置

### 安装依赖库

In [1]:
%%time
%%capture

!pip install llama-index-readers-feishu-wiki

CPU times: user 5.62 ms, sys: 7.69 ms, total: 13.3 ms
Wall time: 1.92 s


### LLM 和嵌入模型

In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 2.16 s, sys: 231 ms, total: 2.39 s
Wall time: 2.01 s


### 设置飞书id和令牌

In [3]:
%%time

app_id = "cli_a644510a88eb900c"
app_secret = "HVk6lCNTQEg22zJkNp7rhehivBUz4GOn"

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.34 µs


In [20]:
%%time

# 通过 https://open.feishu.cn/api-explorer 获取并设置

user_access_token="u-cS3YF1DXlfLUep3XoSB_9Q50gzTNk579UE00508y08CR"
access_token="t-g1048tfhMZ4BKNUOX5ACBZ56BFABXG7IZGVTULHG"

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.81 µs


## 知识库授权

In [21]:
%%time

def get_access_token(app_id, app_secret):
    url = "https://open.feishu.cn/open-apis/auth/v3/tenant_access_token/internal/"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "app_id": app_id,
        "app_secret": app_secret
    }
    response = requests.post(url, headers=headers, json=data)
    response_data = response.json()
    if response_data.get("code") == 0:
        return response_data.get("tenant_access_token")
    else:
        raise Exception("Failed to get access token: {}".format(response_data))

access_token=get_access_token(app_id, app_secret)
access_token

CPU times: user 4.6 ms, sys: 61 µs, total: 4.66 ms
Wall time: 137 ms


't-g1048tfhMZ4BKNUOX5ACBZ56BFABXG7IZGVTULHG'

### 获取应用的open_id

In [22]:
%%time

def get_bot_info(access_token):
    url = "https://open.feishu.cn/open-apis/bot/v3/info"
    headers = {
        "Authorization": "Bearer {}".format(access_token),
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response_data.get("code") == 0:
        # return response_data.get("data", {})
        return response_data
    else:
        raise Exception("Failed to get info: {}".format(response_data))

open_id=get_bot_info(access_token)['bot']['open_id']
open_id

CPU times: user 4.7 ms, sys: 74 µs, total: 4.77 ms
Wall time: 210 ms


'ou_4d2f4ed816ff2b3c95ce8304ef0a78af'

### 获取知识库 space_id

In [15]:
%%time

def get_knowledge_bases(access_token):
    url = "https://open.feishu.cn/open-apis/wiki/v2/spaces"
    headers = {
        "Authorization": "Bearer {}".format(access_token),
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response_data.get("code") == 0:
        return response_data.get("data", {}).get("items", [])
    else:
        raise Exception("Failed to get knowledge bases: {}".format(response_data))

space_name="mykb"
space_id = next((item['space_id'] for item in get_knowledge_bases(access_token) if item.get('name') == space_name), None)
space_id

CPU times: user 5.13 ms, sys: 0 ns, total: 5.13 ms
Wall time: 290 ms


'7408151433373335556'

### 应用添加为知识库成员

见： https://open.feishu.cn/document/server-docs/docs/wiki-v2/wiki-qa#a40ad4ca

In [23]:
%%time

def add_knowledge_space_member(user_access_token, space_id, user_id):
    url = f"https://open.feishu.cn/open-apis/wiki/v2/spaces/{space_id}/members/batch_add"
    headers = {
        "Authorization": f"Bearer {app_access_token}",
        "Content-Type": "application/json; charset=utf-8"
    }
    data = {
        "member_ids": [user_id],
        "member_type": "admin"
    }

    response = requests.post(url, json=data, headers=headers)
    response_data = response.json()

    if response_data["code"] == 0:
        print(f"Successfully added member {user_id} to space {space_id}")
    else:
        raise Exception(f"Failed to add member: {response_data}")


add_knowledge_space_member(user_access_token, space_id, open_id)

JSONDecodeError: Extra data: line 1 column 5 (char 4)